In [5]:
from transformers import AutoModel, AutoTokenizer
from serving_src.model import SimCSEModel
model = SimCSEModel.from_pretrained("daekeun-ml/KoSimCSE-supervised-roberta-large") 
tokenizer = AutoTokenizer.from_pretrained("daekeun-ml/KoSimCSE-supervised-roberta-large") 

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from src.infer import show_embedding_score
sentences = ['이번 주 일요일에 분당 이마트 점은 문을 여나요?',
             '일요일에 분당 이마트는 문 열어요?',
             '분당 이마트 점은 토요일에 몇 시까지 하나요']
show_embedding_score(tokenizer, model.cpu(), sentences)

tensor([[92.9861]], grad_fn=<MulBackward0>) tensor([[83.0575]], grad_fn=<MulBackward0>)


In [7]:
import torch
from typing import List
from src.sts import STSEvaluation
from more_itertools import chunked
batch_size = 128
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

@torch.inference_mode()
def encode(texts: List[str]) -> torch.Tensor:
    embs = []
    for text in chunked(texts, batch_size):
        batch: BatchEncoding = tokenizer(
            text,
            padding=True,
            truncation=True,
            return_tensors="pt",
        )
        emb = model(**batch.to(device))#, use_mlp=False)
        embs.append(emb.cpu())
    return torch.cat(embs, dim=0)

evaluation = STSEvaluation()
sts_metrics = evaluation(encode=encode)

In [8]:
display(sts_metrics)

{'kluests': {'avg': 85.37528732765182,
  'cosine_pearson': 84.38099504950435,
  'cosine_spearman': 85.99332894481987,
  'euclidean_pearson': 85.97399708011103,
  'euclidean_spearman': 85.81717953035523,
  'manhattan_pearson': 86.00645758238008,
  'manhattan_spearman': 85.79909839437714,
  'dot_pearson': 83.87166724324983,
  'dot_spearman': 85.15957479641696},
 'korsts': {'avg': 85.54992865436176,
  'cosine_pearson': 85.41847400009277,
  'cosine_spearman': 85.78030194600866,
  'euclidean_pearson': 85.18741301134143,
  'euclidean_spearman': 85.51742862510163,
  'manhattan_pearson': 85.26303943156756,
  'manhattan_spearman': 85.61542163893404,
  'dot_pearson': 85.7090813794869,
  'dot_spearman': 85.90826920236108}}